In [ ]:
import torch
from transformers import AutoModelForCausalLM
from specdecodes.models.draft_models.eagle_sd import EagleSDDraftModel

target_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map='meta')
draft_model = EagleSDDraftModel(target_model=target_model)

Convert weights

In [ ]:
model_state_dict = torch.load('pytorch_model.bin', map_location='cpu')
new_state_dict = {}
for key, value in model_state_dict.items():
    if key.startswith('layers.'):
        # add models prefix to the key
        new_key = f'model.{key}'
        new_state_dict[new_key] = value
    elif key.startswith("fc"):
        # add models prefix to the key
        new_key = f'fusion.{key}'
        new_state_dict[new_key] = value
    elif key.startswith('embed'):
        # skip the embedding layers
        continue
    else:
        new_state_dict[key] = value
model_state_dict = new_state_dict

draft_model.load_state_dict(model_state_dict)

Save weights to directory

In [ ]:
import os
from safetensors.torch import save_model

save_dir = '/home/scott306lr_l/checkpoints/eagle/official/EAGLE-LLaMA3-Instruct-8B/'

# create folder if not exists
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_model(draft_model, os.path.join(save_dir, 'model.safetensors'))